In [ ]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
data = input_data.read_data_sets("/tmp/data/", one_hot=True)

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [8]:
lr = 0.01
batch_size = 64
display_epochs = 1
logs_path = '/tmp/tensorflow_logs/mnist_fully_connected'
num_steps = int(data.train.num_examples / batch_size)
num_epochs = 5
wh = 28
config = tf.ConfigProto(device_count = {'GPU': 1})

inp = wh*wh
out = num_classes = 10

X = tf.placeholder(tf.float32, shape=[None, inp], name="InputData")
Y = tf.placeholder(tf.float32, shape=[None, out], name="LabelsData")
dropout_prob = tf.placeholder(tf.float32) # dropout prob for test/validation

In [9]:
def conv2d(x, W, b, stride=1, padding="SAME"):
    x = tf.nn.conv2d(x, W, strides=[1, stride, stride, 1], padding=padding)
    x = tf.nn.bias_add(x, b)
    x = tf.nn.relu(x)
    return x


def max_pool(x, stride=2, padding="SAME"):
    x = tf.nn.max_pool(x, ksize=[1, stride, stride, 1], strides=[1, stride, stride, 1], padding=padding)
    return x

In [10]:
W = {
    "c1": tf.Variable(tf.random_normal([5, 5, 1, 32]), name="Weights_Conv1"),
    "c2": tf.Variable(tf.random_normal([5, 5, 32, 64]), name="Weights_Conv2"),
    "fc1": tf.Variable(tf.random_normal([7*7*64, 1024]), name="Weights_FC1"),
    "out": tf.Variable(tf.random_normal([1024, num_classes]), name="Weights_Output"),
}

b = {
    "c1": tf.Variable(tf.random_normal([32]), name="Bias_Conv1"),
    "c2": tf.Variable(tf.random_normal([64]), name="Bias_Conv2"),
    "fc1": tf.Variable(tf.random_normal([1024]), name="Bias_FC1"),
    "out": tf.Variable(tf.random_normal([num_classes]), name="Bias_Output"),
}

In [12]:
def nnet(x, W, b, dropout):
    x = tf.reshape(x, shape=[-1, 28, 28, 1])
    c1 = conv2d(x, W["c1"], b["c1"], stride=1, padding="SAME")
    c1 = max_pool(c1, stride=2)

    c2 = conv2d(c1, W["c2"], b["c2"], stride=1, padding="SAME")
    c2 = max_pool(c2, stride=2)

    fc1 = tf.reshape(c2, [-1, W['fc1'].get_shape().as_list()[0]])
    fc1 = tf.add(tf.matmul(fc1, W["fc1"]), b["fc1"])
    fc1 = tf.nn.relu(fc1)

    fc1 = tf.nn.dropout(fc1, dropout_prob) # dropout

    nn = out_layer = tf.add(tf.matmul(fc1, W['out']), b['out'])
    return nn

In [13]:
nn = nnet(X, W, b, dropout_prob)
with tf.name_scope('Loss'):
    loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
        logits=nn, labels=Y
    ))

with tf.name_scope('Adam'):
    train_op = tf.train.AdamOptimizer(lr).minimize(loss_op)

with tf.name_scope('Accuracy'):
    accuracy = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(nn, 1), tf.argmax(Y, 1)), tf.float32))

predictions = tf.argmax(nn, 1)

init = tf.global_variables_initializer()


training_loss = tf.summary.scalar("Training_Loss", loss_op)

validation_accuracy = tf.summary.scalar("Validation_Accuracy", accuracy)
validation_loss = tf.summary.scalar("Validation_Loss", loss_op)

test_accuracy = tf.summary.scalar("Test_Accuracy", accuracy)

training_summary_op = tf.summary.merge([training_loss])
test_summary_op = tf.summary.merge([test_accuracy])
validation_summary_op = tf.summary.merge([validation_loss, validation_accuracy])
preds = []

In [20]:
with tf.Session(config=config) as S:
    train_writer = tf.summary.FileWriter(logs_path + '/train', S.graph)
    validation_writer = tf.summary.FileWriter(logs_path + '/validation')
    test_writer = tf.summary.FileWriter(logs_path + '/test')

    S.run(init)
    for epoch in range(1, num_epochs + 1):
        avg_cost = 0.
        for step in range(1, num_steps + 1):
            batchX, batchY = data.train.next_batch(batch_size)
            _, c, summary =  S.run([train_op, loss_op, training_summary_op], feed_dict= { X: batchX, Y: batchY, dropout_prob: 0.75 })
            avg_cost += c / num_steps
            train_writer.add_summary(summary, epoch * num_steps + step)

        vall_acc, val_loss, summary = S.run([accuracy, loss_op, validation_summary_op], feed_dict= { X: data.validation.images, Y: data.validation.labels, dropout_prob: 1.0 })
        validation_writer.add_summary(summary, epoch)

        if epoch % display_epochs == 0 or epoch == 1:
            print("Epoch: ", '%04d' % epoch, "Cost: ", "{:.5f}".format(avg_cost), "Validation Accuracy: ", vall_acc)

    print("Optimization Finished!")
    # not all images used due to resource issues
    test_acc, preds, summary = S.run([accuracy, predictions, test_summary_op], feed_dict= { X: data.test.images[:5000], Y: data.test.labels[:5000], dropout_prob: 1.0 })
    test_writer.add_summary(summary, 1)
    print("Test Accuracy:", test_acc)


Epoch:  0001 Cost:  979.88418 Validation Accuracy:  0.971
Epoch:  0002 Cost:  48.08809 Validation Accuracy:  0.9716
Epoch:  0003 Cost:  28.33201 Validation Accuracy:  0.9786
Epoch:  0004 Cost:  18.80928 Validation Accuracy:  0.978
Epoch:  0005 Cost:  14.45312 Validation Accuracy:  0.9798
Optimization Finished!
Test Accuracy: 0.9746


In [22]:
# Run tensorboard --logdir=/tmp/tensorflow_logs
from visualization import create_atlas, create_facets
import numpy as np
from IPython.core.display import display, HTML

with tf.Session() as S:
    actual = np.array(tf.argmax(data.test.labels[:5000], 1).eval())

facets = create_facets(data.test.images, preds, actual, "mnist", "convolutional")

display(HTML(facets))